# Sieci neuronowe i Deep Learning
# Temat 6: PyTorch dla bardziej zaawansowanych

## Zadanie 6.1

Na przykładzie problemu XOR porównać działanie sieci MLP o różnych architekturach (o różnej głębokości i szerokości).

- Przeanalizować, co się dzieje, gdy dokładamy kolejne warstwy ukryte (jedną, dwie, trzy i cztery) o tej samej wielkości, na przykład z 4, 10 neuronami.

- Przeanalizować, co się dzieje, gdy zwiększamy szerokość ukrytych warstw (dla jednej ukrytej warstwy i dla dwóch ukrytych warstw).

- Przy której ze strategii będziemy w stanie osiągać dobre wyniki predykcyjne dla możliwie małej (o małej liczbie parametrów) sieci neuronowej?

## Zadanie 6.2*

Wykorzystując klasę `nn.Module`,
zdefiniować niestandardową warstwę, która mnoży dane wejściowe przez stałą wartość (skalowanie), która jest (uczonym) parametrem modelu.

Zmodyfikować wybraną sieć (z tych zbudowanych podczas jednych z poprzednich zajęć), uwzględniając w niej zdefiniowaną warstwę.